In [1]:
import torch
import numpy as np
import pandas as pd
import os

if os.getcwd() != '/home/hyeryungson/mucoco':
    print("change dir to '/home/hyeryungson/mucoco'")
    os.chdir('/home/hyeryungson/mucoco')

from notebooks.energy_model_retrain.load_ckpt import define_model

In [2]:
test = pd.read_json('/home/hyeryungson/mucoco/data/toxicity/jigsaw-unintended-bias-in-toxicity-classification/fine-grained/test.jsonl',
                    lines=True)

In [3]:
## select toxic 0.5~1.0
test = test.loc[test['toxicity']>=0.5]
test = test.reset_index(drop=True)
test['toxicity_bin'] = pd.cut(test['toxicity'], np.arange(0.5, 1.1, 0.1), labels=np.arange(6))
test_=test.copy()

## randomly select from each bin
tot_count=150
bin_count=6
test = test.groupby('toxicity_bin').sample(int(tot_count/bin_count), random_state=11)

In [4]:
## add 4 examples from toxicity 1
test = pd.concat([test, test_.loc[((~test_.index.isin(test.index)) & (test_.toxicity>0.99))].sample(4, random_state=11)], axis=0)

In [5]:
test.to_json('./notebooks/results/labelling_samples_test_geq0.5_150.jsonl', orient='records', lines=True)

In [6]:
model,config,tokenizer=define_model('/home/hyeryungson/mucoco/models_bak_contd/roberta-base-jigsaw-toxicity-classifier-with-gpt2-large-embeds/checkpoint_best/pytorch_model.bin',
                                 output_attentions=True, output_hidden_states=True)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializin

DEVICE:  cuda


In [7]:
tokenized=tokenizer(test['text'].tolist(),padding=True, return_tensors="pt", truncation=True)

In [8]:
config.max_position_embeddings

514

In [9]:
# model input size와 같아야할거같은데!
tokenized_ids=tokenized['input_ids']

In [10]:
res = []
for i in range(tokenized_ids.shape[0]):
    tmp_sent = tokenized_ids[i]
    tmp_toks = [tokenizer.decode(x) for x in tmp_sent]
    res.append(tmp_toks)

In [11]:
if os.path.exists('./notebooks/results/toxic_labels.txt'):
    with open('./notebooks/results/toxic_labels.txt','r') as f:
        contents = f.readlines()
    last_index = len(contents)/2
    print(f'starting from index {last_index}')
else:
    fptr = open('./notebooks/results/toxic_labels.txt', 'w')
    fptr.close()
    last_index=0

starting from index 4.0


In [49]:
for i in range(last_index, tokenized_ids.shape[0]):
    try:
        print(f"{test['toxicity'].tolist()[i]} ||||||||||| {''.join(res[i])}")
        for j in range(len(res[i])):
            if res[i][j] != '<|endoftext|>':
                print(f"{j} ------ {res[i][j]}")
        toxic_tokens=input("input comma separated list of tokens that are toxic >>")
        if '-' in toxic_tokens:
            start, end = toxic_tokens.split('-')    
            toxic_tokens = ','.join([str(x) for x in range(int(start.strip()), int(end.strip())+1,1)])
        print(f"List you typed: {toxic_tokens}")
        # toxic_tokens=toxic_tokens.split(",")
        fptr = open('./notebooks/results/toxic_labels.txt', 'a')
        fptr.write(''.join(res[i]) + '\n')
        fptr.write(toxic_tokens + '\n')
        fptr.close()
    except KeyboardInterrupt:
        fptr = open('./notebooks/results/toxic_labels.txt', 'a')
        fptr.write(i + '\n')
        fptr.close()
        last_index = i    
    

0.525 ||||||||||| as long as you cannot admit who your enemies are you are losing fast. these guys want you dead yet they get modern missiles from your army. WTF!!!!<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|e